Задание 2

Вам необходимо построить модель, которая на основании данных, поступающих каждую минуту, определяют качество продукции, производимое на обжиговой машине.

Обжиговая машина представляет собой агрегат, состоящий из 5 одинаковых по размеру камер, в каждой камере установлено по 3 датчика температур. Кроме этого, для данной задачи Вы собрали данные о высоте слоя сырья и его влажности. Высота слоя и влажность измеряются при входе сырья в машину. Сырье проходит через обжиговую машину за час.

Данные с показателями работы обжиговой машины содержатся в файле X_data.csv.

Качество продукции измеряется в лаборатории по пробам, которые забираются каждый час, данные по известным анализам содержатся в файле Y_train.csv. В файле указано время забора пробы, проба забирается на выходе из обжиговой машины.


Вы договорились с заказчиком, что оценкой модели будет являться показатель MAE, для оценки модели необходимо сгенерировать предсказания за период, указанный в файле Y_submit.csv (5808 предиктов).



In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor, cv, Pool

%matplotlib inline

In [2]:
df = pd.read_csv('X_data.csv', sep=';', index_col=0, parse_dates=True)
df.head(3)

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22


In [17]:
# Признаки и метки для обучения

Y = pd.read_csv('Y_train.csv', sep=';', index_col=0, header=None, parse_dates=True)
X = df[df.index.isin(Y.index)]
X.shape, Y.shape

((29184, 17), (29184, 1))

In [4]:
Y.head()

,1
0,
2015-01-04 00:05:00,392
2015-01-04 01:05:00,384
2015-01-04 02:05:00,393
2015-01-04 03:05:00,399
2015-01-04 04:05:00,400


Разобьем обучающую выборку на 2 части в пропорции 7/3 не перемешивая.

In [18]:
train_share = int(.7 * X.shape[0])
x_train, y_train = X.iloc[:train_share, :], Y.iloc[:train_share]
x_valid, y_valid  = X.iloc[train_share:, :], Y.iloc[train_share:]
x_train.shape, x_valid.shape

((20428, 17), (8756, 17))

In [19]:
# Признаки и метки для предикта на итоговой модели

y_submit = pd.read_csv('Y_submit.csv', sep=';', index_col=0, header=None, parse_dates=True)
x_submit = df[df.index.isin(y_submit.index)]
y_submit.shape, x_submit.shape

((5808, 1), (5808, 17))

### Исследование признаков

Все признаки являются вещественными. Пропусков нет. Таблица корреляции показывает линейную зависимость между показаниями датчиков, установленных в одной камере, что закономерно.

In [89]:
X.corr()

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
T_data_1_1,1.000000,0.659668,0.649625,-0.005235,0.000708,0.000122,-0.010778,-0.007205,-0.009579,0.000628,-0.016972,-0.005507,-0.005560,-0.006180,-0.015449,-0.019331,-0.001418
T_data_1_2,0.659668,1.000000,0.690438,0.008710,0.009286,0.011731,-0.002342,-0.013459,-0.011845,-0.001686,-0.020158,-0.007527,-0.014022,-0.018872,-0.023108,-0.017738,-0.000291
T_data_1_3,0.649625,0.690438,1.000000,0.002835,0.007235,0.011749,-0.006373,-0.007384,-0.010108,0.005209,-0.003454,0.000794,-0.021769,-0.030411,-0.035314,-0.012020,-0.002702
T_data_2_1,-0.005235,0.008710,0.002835,1.000000,0.355370,0.389946,-0.010062,-0.007176,-0.004971,0.021294,0.010942,0.025101,0.002258,-0.009570,-0.003175,-0.001611,-0.001315
T_data_2_2,0.000708,0.009286,0.007235,0.355370,1.000000,0.404360,-0.023292,-0.000592,-0.017482,0.008122,-0.005910,0.014449,0.010317,0.008459,0.008991,0.013029,0.002594
T_data_2_3,0.000122,0.011731,0.011749,0.389946,0.404360,1.000000,-0.000805,-0.000372,0.001864,0.010093,0.006046,0.013371,0.010390,-0.000028,0.011401,0.008324,0.002564
T_data_3_1,-0.010778,-0.002342,-0.006373,-0.010062,-0.023292,-0.000805,1.000000,0.527216,0.558841,-0.025132,-0.025089,-0.015367,0.007238,0.008074,0.003190,0.018755,-0.005755
T_data_3_2,-0.007205,-0.013459,-0.007384,-0.007176,-0.000592,-0.000372,0.527216,1.000000,0.540306,-0.018672,-0.031407,-0.019870,0.005811,0.001764,0.008637,0.013194,0.001493
T_data_3_3,-0.009579,-0.011845,-0.010108,-0.004971,-0.017482,0.001864,0.558841,0.540306,1.000000,-0.010582,-0.012529,-0.011642,-0.009149,0.003999,-0.001516,0.010508,0.001088
T_data_4_1,0.000628,-0.001686,0.005209,0.021294,0.008122,0.010093,-0.025132,-0.018672,-0.010582,1.000000,0.414581,0.421842,-0.005767,-0.006066,0.001401,-0.011916,0.000479


Обучим и SGD регрессию, ансамбль из деревьев, а так же метрическую модель - SVM. Попробуем метод ближайших соседей - kNN 

SGD-регрессор

In [34]:
# Признаки масштабируем, оцениваем модель на 5-фолдах кроссвалидации с перемешиванием,
#в качестве метрики - MAE.

cross_val_results = []
sgdmodel = SGDRegressor(random_state=32)
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('estimator', sgdmodel)])
kf = KFold(n_splits = 3, shuffle=True, random_state=32)
for train_indices, test_indices in kf.split(x_train):
    pipeline.fit(x_train.values[train_indices], np.ravel(y_train)[train_indices])
    cross_val_results.append(mean_absolute_error(pipeline.predict(x_train.values[test_indices]), 
                                                 np.ravel(y_train)[test_indices]))

print('Средняя абсолютная ошибка на кросс-валидации: %f' % np.mean(cross_val_results))

Средняя абсолютная ошибка на кросс-валидации: 15.505869


In [35]:
pipeline.fit(x_train.values, np.ravel(y_train))
print('Средняя абсолютная ошибка на отложенной выборке: %f' 
      % mean_absolute_error(pipeline.predict(x_valid), y_valid))

Средняя абсолютная ошибка на отложенной выборке: 16.534676


In [36]:
# Предсказанные метки

pipeline.predict(X.values[test_indices])

array([354.74456372, 372.22854195, 401.3326712 , ..., 477.25866122,
       483.71824358, 465.97613312])

### SVM

In [ ]:
cross_val_results = []
svm_model = LinearSVC(random_state=32)
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('estimator', svm_model)])
kf = KFold(n_splits = 3, shuffle=True, random_state=32)
for train_indices, test_indices in kf.split(X):
    pipeline.fit(X.values[train_indices], np.ravel(Y)[train_indices])
    cross_val_results.append(mean_absolute_error(pipeline.predict(X.values[test_indices]), 
                                                 np.ravel(Y)[test_indices]))

print('Средняя абсолютная ошибка на кросс-валидации: %f' % np.mean(cross_val_results))

### kNN

In [30]:
# Признаки масштабируем, оцениваем модель на 5-фолдах кроссвалидации, в качестве метрики - MAE.

cross_val_results = []
knnmodel = KNeighborsRegressor()
scaler = StandardScaler()
knnpipeline = Pipeline([('scaler', scaler), ('estimator', knnmodel)])
kf = KFold(n_splits = 3, shuffle=True, random_state=32)
for train_indices, test_indices in kf.split(x_train):
    knnpipeline.fit(x_train.values[train_indices], np.ravel(y_train)[train_indices])
    cross_val_results.append(mean_absolute_error(knnpipeline.predict(x_train.values[test_indices]), 
                                                 np.ravel(y_train)[test_indices]))

print('Средняя абсолютная ошибка на кросс-валидации: %f' % np.mean(cross_val_results))

Средняя абсолютная ошибка на кросс-валидации: 13.985298


In [31]:
knnpipeline.fit(x_train.values, np.ravel(y_train))
print('Средняя абсолютная ошибка на отложенной выборке: %f' 
      % mean_absolute_error(knnpipeline.predict(x_valid), y_valid))

Средняя абсолютная ошибка на отложенной выборке: 15.307903


### Gradien boosting with Catboost

In [22]:
cross_val_results = []
tb = CatBoostRegressor(random_seed=32, eval_metric = 'MAE', verbose = 200, iterations=1200)
scaler = StandardScaler()
tb.fit(x_train, np.ravel(y_train), early_stopping_rounds = 500, 
               eval_set=(x_valid, y_valid),
              use_best_model=True)

0:	learn: 36.6926707	test: 37.6361066	best: 37.6361066 (0)	total: 57.2ms	remaining: 1m 8s
200:	learn: 10.8435413	test: 11.2098681	best: 11.2098681 (200)	total: 10.9s	remaining: 54.3s
400:	learn: 10.1973406	test: 10.6811349	best: 10.6811349 (400)	total: 21.6s	remaining: 43s
600:	learn: 9.9940472	test: 10.5486576	best: 10.5486576 (600)	total: 32.2s	remaining: 32.1s
800:	learn: 9.8319966	test: 10.4491306	best: 10.4491306 (800)	total: 43.7s	remaining: 21.7s
1000:	learn: 9.7130614	test: 10.3869847	best: 10.3869683 (999)	total: 54.6s	remaining: 10.8s
1199:	learn: 9.6170642	test: 10.3461689	best: 10.3461577 (1198)	total: 1m 4s	remaining: 0us

bestTest = 10.34615772
bestIteration = 1198

Shrink model to first 1199 iterations.


In [23]:
print('Средняя абсолютная ошибка на отложенной выборке: %f' 
      % mean_absolute_error(tb.predict(x_valid), y_valid))

Средняя абсолютная ошибка на отложенной выборке: 10.346158


In [24]:
cross_val_results = []
tb = CatBoostRegressor(random_seed=32, eval_metric = 'MAE', verbose = 200)
scaler = StandardScaler()
tbpipeline = Pipeline([('scaler', scaler), ('estimator', tb)])
tbpipeline.fit(x_train, np.ravel(y_train), estimator__early_stopping_rounds = 500, 
               estimator__eval_set=(x_valid, y_valid),
              estimator__use_best_model=True)

0:	learn: 36.6926707	test: 38.5315434	best: 38.5315434 (0)	total: 50.2ms	remaining: 50.2s
200:	learn: 10.8435413	test: 41.4675630	best: 38.5315434 (0)	total: 10.6s	remaining: 42s
400:	learn: 10.1973406	test: 39.4693372	best: 38.5315434 (0)	total: 21.2s	remaining: 31.7s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 38.5315434
bestIteration = 0

Shrink model to first 1 iterations.


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('estimator',
                 <catboost.core.CatBoostRegressor object at 0x000001ABC03EC7F0>)],
         verbose=False)

In [25]:
print('Средняя абсолютная ошибка на отложенной выборке: %f' 
      % mean_absolute_error(tbpipeline.predict(x_valid), y_valid))

Средняя абсолютная ошибка на отложенной выборке: 37.636107


In [15]:
params = {'loss_function':'MAE',
         'verbose':200,
         'random_seed':32,
          'scale':'True'
         }
pool = Pool(x_train, y_train)
scores = cv(pool=pool,
            params=params,
           fold_count=5,
           seed=32,
           shuffle=True,
           )

CatBoostError: c:/goagent/pipelines/buildmaster/catboost.gittt/catboost/libs/options/plain_options_helper.cpp:510: Unknown option {scale} with value "True"